# Importing

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as model
import torch.optim as optim
import copy
import zipfile


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Loading Dataset

In [ ]:
!unzip "./drive/My Drive/gender_data.zip" -d "./data" 

Streaming output truncated to the last 5000 lines.
  inflating: ./data/gender_data/train/male/236__M_Right_middle_finger.BMP  
  inflating: ./data/gender_data/train/male/233__M_Right_middle_finger.BMP  
  inflating: ./data/gender_data/train/male/98__M_Left_ring_finger_Zcut.BMP  
  inflating: ./data/gender_data/train/male/262__M_Right_middle_finger.BMP  
  inflating: ./data/gender_data/train/male/90__M_Left_ring_finger_Zcut.BMP  
  inflating: ./data/gender_data/train/male/51__M_Right_middle_finger.BMP  
  inflating: ./data/gender_data/train/male/96__M_Left_ring_finger_Zcut.BMP  
  inflating: ./data/gender_data/train/male/271__M_Left_thumb_finger.BMP  
  inflating: ./data/gender_data/train/male/192__M_Right_ring_finger.BMP  
  inflating: ./data/gender_data/train/male/211__M_Right_little_finger.BMP  
  inflating: ./data/gender_data/train/male/148__M_Left_index_finger.BMP  
  inflating: ./data/gender_data/train/male/330__M_Left_middle_finger.BMP  
  inflating: ./data/gender_data/train/male

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.ImageFolder('./data/gender_data/train',transform=transform)
testset = torchvision.datasets.ImageFolder('./data/gender_data/test',transform = transform)

In [ ]:
trainset.class_to_idx

{'female': 0, 'male': 1}

In [ ]:
testset.class_to_idx

{'female': 0, 'male': 1}

In [ ]:
torch.unique(torch.tensor(trainset.targets), return_counts=True)

(tensor([0, 1]), tensor([3555, 3004]))

In [ ]:
torch.unique(torch.tensor(testset.targets), return_counts=True)

(tensor([0, 1]), tensor([1181, 1009]))

In [ ]:
!ls -a './data/SocoFing_data(alter_real_and_gender_train_test)/finger_data_alter_real_test'

.  ..  Altered	Real


In [ ]:
!rm -r './data/SocoFing_data(alter_real_and_gender_train_test)/finger_data_alter_real_test/.ipynb_checkpoints'

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 32,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size = 32,shuffle=False)

# Architectures

## Inception_v3 Architecture

In [ ]:
inception = model.inception_v3(pretrained= True)
print(inception)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth



Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), str

In [ ]:
for param in inception.parameters():
  param.requires_grad = False

In [ ]:
aux_in_features = inception.AuxLogits.fc.in_features
inception.AuxLogits.fc = nn.Linear(aux_in_features, 2)

In [ ]:
for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([2, 768])
torch.Size([2])


In [ ]:
in_features = inception.fc.in_features
inception.fc = nn.Linear(in_features, 2)

In [ ]:
for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([2, 768])
torch.Size([2])
torch.Size([2, 2048])
torch.Size([2])


In [ ]:
# for param in inception.Conv2d_1a_3x3.parameters():
#   param.requires_grad = True


In [ ]:
for param in inception.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([2, 768])
torch.Size([2])
torch.Size([2, 2048])
torch.Size([2])


In [ ]:
def evaluation_inception(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs, aux_outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [ ]:
transform = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.ImageFolder('./data/gender_data/train',transform=transform)
testset = torchvision.datasets.ImageFolder('./data/gender_data/test',transform = transform)

In [ ]:
inception = inception.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(inception.parameters(), lr=0.0001)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 32,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size = 32,shuffle=False)


In [ ]:
loss_epoch_arr = []
max_epochs = 25

min_loss = 1000

#n_iters = np.ceil(50000/batch_size)

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs, aux_outputs = inception(inputs)
        loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_outputs, labels)
        loss.backward()
        opt.step()
        
        if min_loss > loss.item():
            min_loss = loss.item()
            best_model = copy.deepcopy(inception.state_dict())
            #print('Min loss %0.2f' % min_loss)
        
        if i % 400 == 0:
            print('Iteration: %d, Loss: %0.2f' % (i,  loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())
  
    
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
    epoch, max_epochs, 
    evaluation_inception(testloader, inception), 
    evaluation_inception(trainloader, inception)))

    
plt.plot(loss_epoch_arr)
plt.show()

Iteration: 0, Loss: 1.01
Epoch: 0/25, Test acc: 52.74, Train acc: 63.77
Iteration: 0, Loss: 0.83
Epoch: 1/25, Test acc: 51.14, Train acc: 69.71
Iteration: 0, Loss: 0.67


## AlexNet

In [ ]:
alex_net = model.alexnet(pretrained=True)
print(alex_net)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)


In [ ]:
for param in alex_net.parameters():
  param.requires_grad = False

  

In [ ]:
in_features = alex_net.classifier[6].in_features
alex_net.classifier[6] = nn.Linear(in_features,2)

In [ ]:
for param in alex_net.parameters():
  if(param.requires_grad ):
    print(param.size())

  

torch.Size([2, 4096])
torch.Size([2])


In [ ]:
print(alex_net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
for param in alex_net.features[3:9].parameters():
  param.requires_grad = True


In [ ]:
for param in alex_net.parameters():
  if (param.requires_grad ):
    print(param.size())

  

torch.Size([192, 64, 5, 5])
torch.Size([192])
torch.Size([384, 192, 3, 3])
torch.Size([384])
torch.Size([256, 384, 3, 3])
torch.Size([256])
torch.Size([2, 4096])
torch.Size([2])


In [ ]:
def fit(epochs,loss_fn,opt,model):

  loss_arr = []
  n_iter = 400
  for epoch in range(epochs):
    
    for i,data in enumerate(trainloader):
      
      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      loss = loss_fn(outputs,labels)
      loss.backward()
      opt.step()
      del inputs,labels,outputs
      torch.cuda.empty_cache()
      if(i % 100 == 0):
        print(f'Epoch:{epoch+1}: Iteration {i}/{n_iter}: Loss: {loss.item()}')
        
    loss_arr.append(loss.item())
    print(f'Epoch:{epoch}/{epochs}: The Train Accuracy: {evaluation(trainloader,model)} . The Test Accuracy: {evaluation(testloader,model)}')
      
  

  plt.plot(loss_arr)
  plt.show()

    
  

In [ ]:
def evaluation(dataloader,model):

    total,correct = 0,0
    for data in dataloader:

      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)

      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      correct += (pred == labels).sum().item()
      total += labels.size(0)

    return 100*correct/total


In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False)
alex_net = alex_net.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(alex_net.parameters(),lr = 0.001)
fit(150,loss_fn,opt,alex_net)

Epoch:1: Iteration 0/400: Loss: 0.7207645177841187
Epoch:0/150: The Train Accuracy: 66.93093459368806 . The Test Accuracy: 62.8310502283105
Epoch:2: Iteration 0/400: Loss: 0.6452995538711548
Epoch:1/150: The Train Accuracy: 68.79097423387712 . The Test Accuracy: 64.24657534246575
Epoch:3: Iteration 0/400: Loss: 0.5950793623924255
Epoch:2/150: The Train Accuracy: 75.59079127915841 . The Test Accuracy: 61.50684931506849
Epoch:4: Iteration 0/400: Loss: 0.48992428183555603
Epoch:3/150: The Train Accuracy: 73.38008842811404 . The Test Accuracy: 61.7351598173516
Epoch:5: Iteration 0/400: Loss: 0.5857285857200623
Epoch:4/150: The Train Accuracy: 76.09391675560299 . The Test Accuracy: 65.52511415525115
Epoch:6: Iteration 0/400: Loss: 0.47715091705322266
Epoch:5/150: The Train Accuracy: 81.00320170757738 . The Test Accuracy: 63.42465753424658
Epoch:7: Iteration 0/400: Loss: 0.45477405190467834


KeyboardInterrupt: ignored

## VGG

In [ ]:
vgg = model.vgg16_bn(pretrained=True)
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 25

In [ ]:
for param in vgg.parameters():
    param.requires_grad = False

In [ ]:
for param in vgg.features[3:12].parameters():
  param.requires_grad = True

In [ ]:
final_in_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(final_in_features, 2)

In [ ]:
for param in vgg.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([128, 128, 3, 3])
torch.Size([128])
torch.Size([128])
torch.Size([128])
torch.Size([2, 4096])
torch.Size([2])


In [ ]:
vgg = vgg.to(device)
#manual_weights = [0.5,3]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(vgg.parameters(), lr=0.0005)
fit(20,loss_fn,opt,vgg)

Epoch:1: Iteration 0/400: Loss: 9.352944374084473
Epoch:1: Iteration 100/400: Loss: 5.228086948394775
Epoch:0/20: The Train Accuracy: 64.68032278088144 . The Test Accuracy: 53.09477756286267
Epoch:2: Iteration 0/400: Loss: 10.78887939453125
Epoch:2: Iteration 100/400: Loss: 11.340219497680664
Epoch:1/20: The Train Accuracy: 63.93544382371198 . The Test Accuracy: 50.67698259187621
Epoch:3: Iteration 0/400: Loss: 3.998809576034546


KeyboardInterrupt: ignored

## ResNet-18

In [ ]:
resnet = model.resnet18(pretrained=True)
print(resnet)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
in_features = resnet.fc.in_features
resnet.fc = nn.Linear(in_features, 2)

In [ ]:
for param in resnet.layer1[0].parameters():
  param.requires_grad = True

In [ ]:
for param in resnet.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([2, 512])
torch.Size([2])


In [ ]:
resnet = resnet.to(device)
#manual_weights = [0.5,3]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(resnet.parameters(), lr=0.00001)
fit(25,loss_fn,opt,resnet)

Epoch:1: Iteration 0/400: Loss: 0.6886680126190186
Epoch:1: Iteration 100/400: Loss: 0.7181981801986694
Epoch:0/25: The Train Accuracy: 57.47982619491 . The Test Accuracy: 51.353965183752415
Epoch:2: Iteration 0/400: Loss: 0.7053114175796509
Epoch:2: Iteration 100/400: Loss: 0.7218310236930847
Epoch:1/25: The Train Accuracy: 59.155803848541275 . The Test Accuracy: 50.09671179883946
Epoch:3: Iteration 0/400: Loss: 0.7426385879516602
Epoch:3: Iteration 100/400: Loss: 0.6178941130638123
Epoch:2/25: The Train Accuracy: 60.30415890751086 . The Test Accuracy: 50.483558994197296
Epoch:4: Iteration 0/400: Loss: 0.6339307427406311
Epoch:4: Iteration 100/400: Loss: 0.6483222246170044
Epoch:3/25: The Train Accuracy: 62.19739292364991 . The Test Accuracy: 49.61315280464217
Epoch:5: Iteration 0/400: Loss: 0.647345781326294
Epoch:5: Iteration 100/400: Loss: 0.6629218459129333
Epoch:4/25: The Train Accuracy: 63.3147113594041 . The Test Accuracy: 49.516441005802704
Epoch:6: Iteration 0/400: Loss: 0.61

KeyboardInterrupt: ignored

# MobileNet

In [ ]:
mobilenet = model.mobilenet_v2(pretrained = True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


In [ ]:
print(mobilenet)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
for param in mobilenet.parameters():
  param.requires_grad = False

  

In [ ]:
in_features = mobilenet.classifier[1].in_features
mobilenet.classifier[1] = nn.Linear(in_features,2)

In [ ]:
for param in mobilenet.parameters():
  if(param.requires_grad ):
    print(param.size())

  

torch.Size([2, 1280])
torch.Size([2])


In [ ]:
for param in mobilenet.features[0:2].parameters():
  param.requires_grad = True


In [ ]:
for param in mobilenet.parameters():
  if (param.requires_grad ):
    print(param.size())

  

torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32, 1, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([16, 32, 1, 1])
torch.Size([16])
torch.Size([16])
torch.Size([2, 1280])
torch.Size([2])


In [ ]:
def fit(epochs,loss_fn,opt,model):

  loss_arr = []
  n_iter = 400
  for epoch in range(epochs):
    
    for i,data in enumerate(trainloader):
      
      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      loss = loss_fn(outputs,labels)
      loss.backward()
      opt.step()
      del inputs,labels,outputs
      torch.cuda.empty_cache()
      if(i % 100 == 0):
        print(f'Epoch:{epoch+1}: Iteration {i}/{n_iter}: Loss: {loss.item()}')
        
    loss_arr.append(loss.item())
    print(f'Epoch:{epoch}/{epochs}: The Train Accuracy: {evaluation(trainloader,model)} . The Test Accuracy: {evaluation(testloader,model)}')
      
  

  plt.plot(loss_arr)
  plt.show()

    
  

In [ ]:
def evaluation(dataloader,model):

    total,correct = 0,0
    for data in dataloader:

      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)

      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      correct += (pred == labels).sum().item()
      total += labels.size(0)

    return 100*correct/total


In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=64,shuffle=False)

mobilenet = mobilenet.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(mobilenet.parameters(),lr = 0.01)
fit(30,loss_fn,opt,mobilenet)

Epoch:1: Iteration 0/400: Loss: 0.691527783870697
Epoch:0/30: The Train Accuracy: 70.2048417132216 . The Test Accuracy: 49.806576402321085
Epoch:2: Iteration 0/400: Loss: 0.5618043541908264


KeyboardInterrupt: ignored

# AkaNet

In [ ]:
class AkaNet(nn.Module):
  
  def __init__(self):
    
    super(AkaNet,self).__init__()

    self.model_cnn = nn.Sequential(
        nn.Conv2d(3,6,3), #(N,3,104,104) --> (N,6,102,102)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2),
        nn.BatchNorm2d(6), #(N,6,102,102) -->(N,6,51,51)
        nn.Conv2d(6,16,5), #(N,6,51,51)  --> (N,16,47,47)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,16,47,47) --> (N,16,23,23)
        nn.BatchNorm2d(16),
        nn.Conv2d(16,8,5), #(N,16,23,23)  --> (N,8,19,19)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,8,19,19) --> (N,8,9,9)
        nn.BatchNorm2d(8),
        
        
    )

    self.model_fc = nn.Sequential(
        nn.Dropout(),
        nn.Linear(648,120),
        nn.ReLU(),
        nn.Linear(120,60),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(60,20),
        nn.ReLU(),
        nn.Linear(20,2) 
    )

  def forward(self,x):
    x = self.model_cnn(x)
    x = x.view(x.size(0),-1)
    x = self.model_fc(x)
    return x


In [ ]:
net = AkaNet()

In [ ]:
def evaluation(dataloader,model):

    total,correct = 0,0
    for data in dataloader:

      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)

      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      correct += (pred == labels).sum().item()
      total += labels.size(0)

    return 100*correct/total


In [ ]:
def fit(epochs,loss_fn,opt,model):

  loss_arr = []
  n_iter = 400
  for epoch in range(epochs):
    
    for i,data in enumerate(trainloader):
      
      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      loss = loss_fn(outputs,labels)
      loss.backward()
      opt.step()
      del inputs,labels,outputs
      torch.cuda.empty_cache()
      if(i % 100 == 0):
        print(f'Epoch:{epoch+1}: Iteration {i}/{n_iter}: Loss: {loss.item()}')
        
    loss_arr.append(loss.item())
    print(f'Epoch:{epoch}/{epochs}: The Train Accuracy: {evaluation(trainloader,model)} . The Test Accuracy: {evaluation(testloader,model)}')
      
  

  plt.plot(loss_arr)
  plt.show()

    
  

In [ ]:
transform = transforms.Compose([
    transforms.Resize((104,104)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.ImageFolder('./data/SocoFing_data(alter_real_and_gender_train_test)/finger_gender_data_train',transform=transform)
testset = torchvision.datasets.ImageFolder('./data/SocoFing_data(alter_real_and_gender_train_test)/finger_gender_data_test',transform = transform)

In [ ]:
trainset.class_to_idx

{'female': 0, 'male': 1}

In [ ]:
testset.class_to_idx

{'female': 0, 'male': 1}

In [ ]:
net = AkaNet()


In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False)
net = AkaNet()
net = net.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(),lr=0.0001)
fit(150,loss_fn,opt,net)

Epoch:1: Iteration 0/400: Loss: 0.6821238994598389
Epoch:0/150: The Train Accuracy: 53.6933581626319 . The Test Accuracy: 49.806576402321085
Epoch:2: Iteration 0/400: Loss: 0.6845964789390564
Epoch:1/150: The Train Accuracy: 54.93482309124767 . The Test Accuracy: 50.77369439071567
Epoch:3: Iteration 0/400: Loss: 0.6891566514968872
Epoch:2/150: The Train Accuracy: 59.74549968963377 . The Test Accuracy: 52.804642166344294
Epoch:4: Iteration 0/400: Loss: 0.6519125699996948
Epoch:3/150: The Train Accuracy: 64.55617628801987 . The Test Accuracy: 56.76982591876209
Epoch:5: Iteration 0/400: Loss: 0.6269930601119995
Epoch:4/150: The Train Accuracy: 66.10800744878958 . The Test Accuracy: 54.642166344294004
Epoch:6: Iteration 0/400: Loss: 0.6036604046821594
Epoch:5/150: The Train Accuracy: 67.008069522036 . The Test Accuracy: 56.479690522243715
Epoch:7: Iteration 0/400: Loss: 0.6077096462249756
Epoch:6/150: The Train Accuracy: 68.1253879577902 . The Test Accuracy: 56.38297872340426
Epoch:8: Iter

KeyboardInterrupt: ignored

# LeNet

In [ ]:
class LeNet(nn.Module):
  
  def __init__(self):
    
    super(LeNet,self).__init__()

    self.model_cnn = nn.Sequential(
        nn.Conv2d(3,6,3), #(N,1,224,224) --> (N,6,222,222)
        nn.ReLU(),
        nn.AvgPool2d(2,stride=2), #(N,6,222,222) -->(N,6,111,111)
        nn.Conv2d(6,16,5), #(N,6,111,111)  --> (N,16,107,107)
        nn.ReLU(),
        nn.AvgPool2d(2,stride=2) #(N,16,107,107) --> (N,16,53,53)
    )

    self.model_fc = nn.Sequential(
        nn.Linear(44944,120),
        nn.ReLU(),
        nn.Linear(120,84), 
        nn.ReLU(),
        nn.Linear(84,2) 
    )

  def forward(self,x):
    x = self.model_cnn(x)
    x = x.view(x.size(0),-1)
    x = self.model_fc(x)
    return x


In [ ]:
def evaluation(dataloader,model):

    total,correct = 0,0
    for data in dataloader:

      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)

      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      correct += (pred == labels).sum().item()
      total += labels.size(0)

    return 100*correct/total


In [ ]:
def fit(epochs,loss_fn,opt,model):

  loss_arr = []
  n_iter = 400
  for epoch in range(epochs):
    
    for i,data in enumerate(trainloader):
      
      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      loss = loss_fn(outputs,labels)
      loss.backward()
      opt.step()
      del inputs,labels,outputs
      torch.cuda.empty_cache()
      if(i % 100 == 0):
        print(f'Epoch:{epoch+1}: Iteration {i}/{n_iter}: Loss: {loss.item()}')
        
    loss_arr.append(loss.item())
    print(f'Epoch:{epoch}/{epochs}: The Train Accuracy: {evaluation(trainloader,model)} . The Test Accuracy: {evaluation(testloader,model)}')
      
  

  plt.plot(loss_arr)
  plt.show()

    
  

In [ ]:
net = LeNet()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.ImageFolder('./data/gender_data/train',transform=transform)
testset = torchvision.datasets.ImageFolder('./data/gender_data/test',transform = transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False)
net = LeNet()
net = net.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.RMSprop(net.parameters(),lr=0.001)
fit(200,loss_fn,opt,net)

Epoch:1: Iteration 0/400: Loss: 0.6986533999443054
Epoch:0/200: The Train Accuracy: 72.52629974081415 . The Test Accuracy: 62.73972602739726
Epoch:2: Iteration 0/400: Loss: 0.5629969239234924
Epoch:1/200: The Train Accuracy: 76.30736392742796 . The Test Accuracy: 63.74429223744292
Epoch:3: Iteration 0/400: Loss: 0.44625192880630493
Epoch:2/200: The Train Accuracy: 72.41957615490166 . The Test Accuracy: 61.87214611872146
Epoch:4: Iteration 0/400: Loss: 0.5113197565078735
Epoch:3/200: The Train Accuracy: 80.24089037963104 . The Test Accuracy: 63.60730593607306
Epoch:5: Iteration 0/400: Loss: 0.5053593516349792


KeyboardInterrupt: ignored

# Custom Parallel Model

In [ ]:
class Custom_1(nn.Module):
  
  def __init__(self):
    
    super(Custom_1,self).__init__()

    self.model_cnn = nn.Sequential(
        nn.Conv2d(3,6,3), #(N,3,224,224) --> (N,6,222,222)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,6,222,222) -->(N,6,111,111)
        nn.Conv2d(6,16,5), #(N,6,111,111)  --> (N,16,107,107)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,16,107,107) --> (N,16,53,53)
    )

    self.model_fc = nn.Sequential(
        nn.Dropout(),
        nn.Linear(44944,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,64),


        

    )

  def forward(self,x):
    x = self.model_cnn(x)
    x = x.view(x.size(0),-1)
    x = self.model_fc(x)
    return x


In [ ]:
class Custom_2(nn.Module):
  
  def __init__(self):
    
    super(Custom_2,self).__init__()

    self.model_cnn = nn.Sequential(
        nn.Conv2d(3,6,3), #(N,3,224,224) --> (N,6,222,222)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,6,222,222) -->(N,6,111,111)
        nn.Conv2d(6,16,5), #(N,6,111,111)  --> (N,16,107,107)
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2), #(N,16,107,107) --> (N,16,53,53)
    )

    self.model_fc = nn.Sequential(
        nn.Dropout(),
        nn.Linear(44944,120),
        nn.ReLU(),
        nn.Linear(120,64),


    )

  def forward(self,x):
    x = self.model_cnn(x)
    x = x.view(x.size(0),-1)
    x = self.model_fc(x)
    return x


In [ ]:
class Ensemble(nn.Module):

  def __init__(self,custom_1,custom_2):
    super(Ensemble,self).__init__()

    self.custom_1 = custom_1
    self.custom_2 = custom_2

    self.model_fc = nn.Sequential(
        nn.Dropout(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(64,32),
        nn.ReLU(),
        nn.Linear(32,2) 
    )
  
  def forward(self,inp):
    x1 = self.custom_1(inp)
    x2 = self.custom_2(inp)
    #print(x1.shape)
    #print(x2.shape)
    x = torch.cat([x1,x2],dim = -1)
    
    #print(x.shape)
    #x = self.model_cnn(x)
    x = x.view(x.size(0),-1)
    #print(x.shape)
    x = self.model_fc(x)
    return x

    

In [ ]:
net1 = Custom_1()
net2 = Custom_2()
model = Ensemble(net1,net2)

In [ ]:
a = torch.randn(16,20,20)

b = torch.randn(16,20,20)
print(a.shape)
print(b.shape)
c = torch.cat([a,b],dim = -1)
print(c.shape)

torch.Size([16, 20, 20])
torch.Size([16, 20, 20])
torch.Size([16, 20, 40])


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    #transforms.RandomApply((transforms.RandomRotation(70),transforms.RandomPerspective(),transforms.RandomVerticalFlip())),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.ImageFolder('./data/gender_data/train',transform=transform)
testset = torchvision.datasets.ImageFolder('./data/gender_data/test',transform = transform)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size = 128,shuffle=False)

In [ ]:
def evaluation(dataloader,model):

    total,correct = 0,0
    for data in dataloader:

      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)

      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      correct += (pred == labels).sum().item()
      total += labels.size(0)

    return 100*correct/total


In [ ]:
def fit(epochs,loss_fn,opt,model):

  loss_arr = []
  n_iter = 400
  for epoch in range(epochs):
    
    for i,data in enumerate(trainloader):
      
      inputs,labels = data
      inputs,labels = inputs.to(device),labels.to(device)
      opt.zero_grad()
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

      loss = loss_fn(outputs,labels)
      loss.backward()
      opt.step()
      del inputs,labels,outputs
      torch.cuda.empty_cache()
      if(i % 100 == 0):
        print(f'Epoch:{epoch+1}: Iteration {i}/{n_iter}: Loss: {loss.item()}')
        
    loss_arr.append(loss.item())
    print(f'Epoch:{epoch}/{epochs}: The Train Accuracy: {evaluation(trainloader,model)} . The Test Accuracy: {evaluation(testloader,model)}')
      
  

  plt.plot(loss_arr)
  plt.show()

    
  

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False)
net1 = Custom_1()
net2 = Custom_2()
net = Ensemble(net1,net2)
net = net.to(device)
#manual_weights = [0.5,5]
#manual_weights = torch.FloatTensor(manual_weights).to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(),lr = 0.001)
fit(30,loss_fn,opt,net)

Epoch:1: Iteration 0/400: Loss: 0.6920679807662964
Epoch:0/30: The Train Accuracy: 64.46104589114194 . The Test Accuracy: 57.945205479452056
Epoch:2: Iteration 0/400: Loss: 0.6420509815216064
Epoch:1/30: The Train Accuracy: 81.3081262387559 . The Test Accuracy: 60.821917808219176
Epoch:3: Iteration 0/400: Loss: 0.39546358585357666
Epoch:2/30: The Train Accuracy: 90.41012349443513 . The Test Accuracy: 62.87671232876713
Epoch:4: Iteration 0/400: Loss: 0.25239795446395874


KeyboardInterrupt: ignored